In [5]:
import csv
from collections import defaultdict

def analyze_log_data(file_path, login_threshold=10):
    ip_tracker = defaultdict(int)
    endpoint_tracker = defaultdict(int)
    failed_attempt_tracker = defaultdict(int)
    
    try:
        with open(file_path, 'r') as log:
            for record in log:
                components = record.split()
                if len(components) < 9:
                    continue
                
                user_ip = components[0]
                requested_resource = components[6]
                response_code = components[8]
                
                ip_tracker[user_ip] += 1
                endpoint_tracker[requested_resource] += 1
                
                if response_code == '401' or 'Invalid credentials' in record:
                    failed_attempt_tracker[user_ip] += 1
    except Exception as e:
        print(f"Error processing log file: {e}")
        return

    sorted_ips = sorted(ip_tracker.items(), key=lambda x: x[1], reverse=True)
    top_endpoint = max(endpoint_tracker.items(), key=lambda x: x[1])
    flagged_ips = {ip: count for ip, count in failed_attempt_tracker.items() if count > login_threshold}
    
    print("IP Address           Request Count")
    for ip, count in sorted_ips:
        print(f"{ip:<20} {count}")
    
    print(f"\nMost Frequently Accessed Endpoint:\n{top_endpoint[0]} (Accessed {top_endpoint[1]} times)")
    
    if flagged_ips:
        print("\nSuspicious Activity Detected:")
        print("IP Address           Failed Login Attempts")
        for ip, count in flagged_ips.items():
            print(f"{ip:<20} {count}")

    with open("log_analysis_results.csv", mode="w", newline="") as output_file:
        writer = csv.writer(output_file)
        writer.writerow(["IP Address", "Request Count"])
        writer.writerows(sorted_ips)
        writer.writerow([])
        writer.writerow(["Most Accessed Endpoint", "Access Count"])
        writer.writerow([top_endpoint[0], top_endpoint[1]])
        writer.writerow([])
        writer.writerow(["IP Address", "Failed Login Count"])
        writer.writerows(flagged_ips.items())

analyze_log_data("C:\\Users\\DELL\\OneDrive\\Desktop\\Sample.log", login_threshold=10)


IP Address           Request Count
203.0.113.5          8
198.51.100.23        8
192.168.1.1          7
10.0.0.2             6
192.168.1.100        5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)
